

**Algoritma:** K-Nearest Neighbors (KNN)  
**Dataset:** IEM_dataset.csv  
**Tujuan:** Merekomendasikan IEM berdasarkan budget, genre, dan karakter suara

---

## 1. Import Library

In [ ]:
# Library untuk data processing
import pandas as pd
import numpy as np

# Library untuk visualisasi
import matplotlib.pyplot as plt
import seaborn as sns

# Library untuk machine learning
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

# Library untuk menyimpan model
import joblib

# Setting untuk visualisasi
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print('✅ Semua library berhasil di-import!')

## 2. Load Dataset

In [ ]:
# Load dataset dari file CSV
df = pd.read_csv('../IEM_dataset.csv')

print(f'📊 Dataset berhasil dimuat!')
print(f'Jumlah data: {len(df)} IEM')
print(f'Jumlah kolom: {len(df.columns)}')
print('\nKolom yang tersedia:')
print(df.columns.tolist())

In [ ]:
# Tampilkan 5 data pertama
df.head()

In [ ]:
# Informasi dataset
df.info()

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Statistik deskriptif
df.describe()

In [ ]:
# Cek missing values
print('Missing Values:')
print(df.isnull().sum())
print('\n✅ Dataset sudah bersih, tidak ada missing values!')

In [ ]:
# Distribusi harga IEM
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df['price'], bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Harga (Rp)')
plt.ylabel('Jumlah IEM')
plt.title('Distribusi Harga IEM')
plt.ticklabel_format(style='plain', axis='x')

plt.subplot(1, 2, 2)
plt.boxplot(df['price'])
plt.ylabel('Harga (Rp)')
plt.title('Boxplot Harga IEM')
plt.ticklabel_format(style='plain', axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Distribusi genre musik
plt.figure(figsize=(10, 6))
genre_counts = df['genre'].value_counts()
plt.bar(genre_counts.index, genre_counts.values, edgecolor='black', alpha=0.7)
plt.xlabel('Genre Musik')
plt.ylabel('Jumlah IEM')
plt.title('Distribusi IEM Berdasarkan Genre')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print('\nJumlah IEM per Genre:')
print(genre_counts)

In [ ]:
# Distribusi tuning
plt.figure(figsize=(10, 6))
tuning_counts = df['tuning'].value_counts()
plt.bar(tuning_counts.index, tuning_counts.values, edgecolor='black', alpha=0.7)
plt.xlabel('Jenis Tuning')
plt.ylabel('Jumlah IEM')
plt.title('Distribusi IEM Berdasarkan Tuning')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print('\nJumlah IEM per Tuning:')
print(tuning_counts)

In [ ]:
# Korelasi antar fitur numerik
plt.figure(figsize=(10, 8))
numeric_cols = ['price', 'bass', 'mid', 'treble', 'soundstage', 'impedance', 'sensitivity']
correlation = df[numeric_cols].corr()
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Korelasi Antar Fitur Numerik')
plt.tight_layout()
plt.show()

## 4. Data Preprocessing

### 4.1 Encoding Genre

**Mengapa menggunakan LabelEncoder?**
- Genre bersifat kategorikal ordinal (ada urutan preferensi)
- KNN bekerja dengan jarak euclidean, sehingga encoding numerik lebih efisien
- Dataset relatif kecil, sehingga LabelEncoder lebih sederhana dibanding OneHotEncoder
- OneHotEncoder akan membuat dimensi fitur terlalu besar untuk dataset kecil

In [ ]:
# Buat copy dataset untuk preprocessing
df_processed = df.copy()

# Encoding genre menggunakan LabelEncoder
label_encoder = LabelEncoder()
df_processed['genre_encoded'] = label_encoder.fit_transform(df_processed['genre'])

print('Genre Encoding:')
genre_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
for genre, code in genre_mapping.items():
    print(f'  {genre} → {code}')

print('\n✅ Genre berhasil di-encode!')

### 4.2 Seleksi Fitur untuk Model

Fitur yang digunakan untuk KNN:
- **price**: Harga IEM (sesuai budget user)
- **bass**: Level bass (1-5)
- **mid**: Level mid (1-5)
- **treble**: Level treble (1-5)
- **genre_encoded**: Genre musik yang di-encode

In [ ]:
# Fitur yang akan digunakan untuk model
feature_columns = ['price', 'bass', 'mid', 'treble', 'genre_encoded']

# Ekstrak fitur
X = df_processed[feature_columns].values

print(f'Shape fitur: {X.shape}')
print(f'Jumlah sampel: {X.shape[0]}')
print(f'Jumlah fitur: {X.shape[1]}')
print(f'\nFitur yang digunakan: {feature_columns}')

### 4.3 Normalisasi Fitur

**Mengapa perlu normalisasi?**
- KNN menggunakan jarak euclidean
- Fitur dengan skala besar (price: jutaan) akan mendominasi fitur kecil (bass: 1-5)
- StandardScaler membuat semua fitur memiliki mean=0 dan std=1
- Hasil prediksi menjadi lebih akurat dan seimbang

In [ ]:
# Normalisasi menggunakan StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print('Sebelum normalisasi:')
print(f'  Mean: {X.mean(axis=0)}')
print(f'  Std: {X.std(axis=0)}')

print('\nSetelah normalisasi:')
print(f'  Mean: {X_scaled.mean(axis=0)}')
print(f'  Std: {X_scaled.std(axis=0)}')

print('\n✅ Fitur berhasil dinormalisasi!')

## 5. Training Model KNN

**Algoritma:** K-Nearest Neighbors (KNN)  
**Jumlah Neighbors:** 5  
**Metric:** Euclidean Distance  

**Mengapa KNN?**
- Cocok untuk sistem rekomendasi berbasis similarity
- Tidak memerlukan training yang kompleks
- Mudah dipahami dan diimplementasikan
- Hasil rekomendasi berdasarkan IEM yang paling mirip

In [ ]:
# Inisialisasi model KNN
# n_neighbors=5 artinya akan mencari 5 IEM terdekat
# metric='euclidean' untuk menghitung jarak antar data point
knn_model = NearestNeighbors(n_neighbors=5, metric='euclidean')

# Training model dengan data yang sudah dinormalisasi
knn_model.fit(X_scaled)

print('✅ Model KNN berhasil di-training!')
print(f'Jumlah neighbors: {knn_model.n_neighbors}')
print(f'Metric: {knn_model.metric}')

## 6. Testing Model dengan Simulasi Input User

In [ ]:
# Fungsi untuk mapping input user ke fitur ML
def map_user_input(budget_range, genre, sound_character):
    """
    Mapping input user ke fitur yang digunakan model
    
    Args:
        budget_range: '< 500k', '500k-1jt', '1jt-2jt', '> 2jt'
        genre: 'Pop', 'Rock', 'EDM', 'Jazz', 'Campuran'
        sound_character: 'Bass kuat', 'Seimbang', 'Detail / Jernih'
    
    Returns:
        dict: Fitur yang siap digunakan untuk prediksi
    """
    
    # Mapping budget
    budget_map = {
        '< 500k': 300000,
        '500k-1jt': 750000,
        '1jt-2jt': 1500000,
        '> 2jt': 3000000
    }
    
    # Mapping karakter suara
    sound_map = {
        'Bass kuat': {'bass': 5, 'mid': 3, 'treble': 3},
        'Seimbang': {'bass': 3, 'mid': 4, 'treble': 3},
        'Detail / Jernih': {'bass': 2, 'mid': 3, 'treble': 5}
    }
    
    # Encode genre
    genre_encoded = label_encoder.transform([genre])[0]
    
    # Buat fitur input
    user_features = {
        'price': budget_map[budget_range],
        'bass': sound_map[sound_character]['bass'],
        'mid': sound_map[sound_character]['mid'],
        'treble': sound_map[sound_character]['treble'],
        'genre_encoded': genre_encoded
    }
    
    return user_features

In [ ]:
# Fungsi untuk mendapatkan rekomendasi
def get_recommendations(budget_range, genre, sound_character, top_n=3):
    """
    Mendapatkan rekomendasi IEM berdasarkan input user
    
    Args:
        budget_range: Range budget user
        genre: Genre musik favorit
        sound_character: Karakter suara yang diinginkan
        top_n: Jumlah rekomendasi (default: 3)
    
    Returns:
        DataFrame: Top N rekomendasi IEM
    """
    
    # Mapping input user ke fitur
    user_features = map_user_input(budget_range, genre, sound_character)
    
    # Konversi ke array dan reshape
    user_array = np.array([[
        user_features['price'],
        user_features['bass'],
        user_features['mid'],
        user_features['treble'],
        user_features['genre_encoded']
    ]])
    
    # Normalisasi input user
    user_scaled = scaler.transform(user_array)
    
    # Prediksi menggunakan KNN
    distances, indices = knn_model.kneighbors(user_scaled, n_neighbors=top_n)
    
    # Ambil rekomendasi dari dataset
    recommendations = df.iloc[indices[0]].copy()
    recommendations['distance'] = distances[0]
    
    return recommendations

### 6.1 Contoh Simulasi 1: Budget Rendah, Genre Pop, Bass Kuat

In [ ]:
print('=' * 80)
print('SIMULASI 1: Budget < 500k, Genre Pop, Karakter Bass Kuat')
print('=' * 80)

recommendations_1 = get_recommendations(
    budget_range='< 500k',
    genre='Pop',
    sound_character='Bass kuat'
)

print('\nTop 3 Rekomendasi IEM:')
for idx, row in recommendations_1.iterrows():
    print(f"\n{recommendations_1.index.get_loc(idx) + 1}. {row['name']}")
    print(f"   Brand: {row['brand']}")
    print(f"   Harga: Rp {row['price']:,.0f}")
    print(f"   Tuning: {row['tuning']}")
    print(f"   Bass: {row['bass']}, Mid: {row['mid']}, Treble: {row['treble']}")
    print(f"   Distance Score: {row['distance']:.4f}")

### 6.2 Contoh Simulasi 2: Budget Menengah, Genre Jazz, Detail/Jernih

In [ ]:
print('=' * 80)
print('SIMULASI 2: Budget 1jt-2jt, Genre Jazz, Karakter Detail/Jernih')
print('=' * 80)

recommendations_2 = get_recommendations(
    budget_range='1jt-2jt',
    genre='Jazz',
    sound_character='Detail / Jernih'
)

print('\nTop 3 Rekomendasi IEM:')
for idx, row in recommendations_2.iterrows():
    print(f"\n{recommendations_2.index.get_loc(idx) + 1}. {row['name']}")
    print(f"   Brand: {row['brand']}")
    print(f"   Harga: Rp {row['price']:,.0f}")
    print(f"   Tuning: {row['tuning']}")
    print(f"   Bass: {row['bass']}, Mid: {row['mid']}, Treble: {row['treble']}")
    print(f"   Distance Score: {row['distance']:.4f}")

### 6.3 Contoh Simulasi 3: Budget Tinggi, Genre Campuran, Seimbang

In [ ]:
print('=' * 80)
print('SIMULASI 3: Budget > 2jt, Genre Campuran, Karakter Seimbang')
print('=' * 80)

recommendations_3 = get_recommendations(
    budget_range='> 2jt',
    genre='Campuran',
    sound_character='Seimbang'
)

print('\nTop 3 Rekomendasi IEM:')
for idx, row in recommendations_3.iterrows():
    print(f"\n{recommendations_3.index.get_loc(idx) + 1}. {row['name']}")
    print(f"   Brand: {row['brand']}")
    print(f"   Harga: Rp {row['price']:,.0f}")
    print(f"   Tuning: {row['tuning']}")
    print(f"   Bass: {row['bass']}, Mid: {row['mid']}, Treble: {row['treble']}")
    print(f"   Distance Score: {row['distance']:.4f}")

## 7. Menyimpan Model dan Preprocessing Objects

Model dan objek preprocessing akan disimpan untuk digunakan di Flask backend

In [ ]:
# Simpan model KNN
joblib.dump(knn_model, '../models/knn_model.pkl')
print('✅ Model KNN disimpan: models/knn_model.pkl')

# Simpan scaler
joblib.dump(scaler, '../models/scaler.pkl')
print('✅ Scaler disimpan: models/scaler.pkl')

# Simpan label encoder
joblib.dump(label_encoder, '../models/label_encoder.pkl')
print('✅ Label Encoder disimpan: models/label_encoder.pkl')

# Simpan dataset yang sudah diproses
df.to_csv('../models/processed_dataset.csv', index=False)
print('✅ Dataset disimpan: models/processed_dataset.csv')

print('\n🎉 Semua model dan preprocessing objects berhasil disimpan!')

## 8. Kesimpulan

### Hasil Training:
- ✅ Dataset berhasil dimuat (46 IEM)
- ✅ EDA menunjukkan distribusi data yang baik
- ✅ Preprocessing berhasil (encoding + normalisasi)
- ✅ Model KNN berhasil di-training
- ✅ Testing menunjukkan hasil rekomendasi yang relevan
- ✅ Model dan preprocessing objects berhasil disimpan

### Fitur yang Digunakan:
1. **price**: Harga IEM (sesuai budget)
2. **bass**: Level bass (1-5)
3. **mid**: Level mid (1-5)
4. **treble**: Level treble (1-5)
5. **genre_encoded**: Genre musik (encoded)

### Mapping Input User:
- **Budget**: < 500k → 300k, 500k-1jt → 750k, 1jt-2jt → 1.5jt, > 2jt → 3jt
- **Bass Kuat**: bass=5, mid=3, treble=3
- **Seimbang**: bass=3, mid=4, treble=3
- **Detail/Jernih**: bass=2, mid=3, treble=5

### Next Steps:
1. Integrasi model ke Flask backend
2. Buat interface web untuk input user
3. Tampilkan hasil rekomendasi dengan gambar IEM dan tuning

---

**Notebook selesai! Lanjut ke implementasi Flask backend.**